In [113]:
import numpy as np
import pandas as pd
import os
import git
from git import RemoteProgress
from git import Repo
from datetime import datetime, date
import matplotlib.pyplot as plt
import seaborn as sns
from time import strptime
%matplotlib inline

In [114]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [115]:
#Spring Framework File 1
#remote_link = "https://github.com/spring-projects/spring-framework"
#local_link = "C:\\Users\\User\\repo\\spring-framework"
#repo = Repo(local_link)
#vcc_commit = "af47a8b79b0fa07ff2c038197a07c09b745802ac"
#affected_files = ["spring-webmvc/src/main/java/org/springframework/web/servlet/ResourceServlet.java"]

# Camel
#remote_link = "https://github.com/apache/camel"
#local_link = "C:\\Users\\User\\repo\\camel"
#repo = Repo(local_link)
#vcc_commit = "3c97afe7bcef40ca44d2f98d5f17133500c8df06"
#affected_files = ["camel-core/src/main/java/org/apache/camel/model/dataformat/JsonDataFormat.java"]

# Tika
remote_link = "https://github.com/apache/tika"
local_link = "C:\\Users\\User\\repo\\tika"
repo = Repo(local_link)
vcc_commit = "de7092125649cd61840e27b1f70d68cfab910d89"
affected_files = ["tika-parsers/src/main/java/org/apache/tika/parser/iptc/IptcAnpaParser.java"]

In [116]:
def removeDuplicates(array):
    new_array = []
    for value in array:
        if value not in new_array:
            new_array.append(value)
    return new_array

In [117]:
show_data = repo.git.show("-s", vcc_commit).splitlines()
for line in show_data:
    print(line)


commit de7092125649cd61840e27b1f70d68cfab910d89
Author: Nick Burch <nick@apache.org>
Date:   Sat Apr 28 16:53:35 2012 +0000

    TIKA-858 Patch from Craig Stires to add support for parsing IPTC ANPA News Wire Feeds
    
    git-svn-id: https://svn.apache.org/repos/asf/tika/trunk@1331794 13f79535-47bb-0310-9956-ffa450edef68


In [118]:
files = repo.git.whatchanged("-1", "--format=oneline", vcc_commit).splitlines()
files.pop(0)
print(str(len(files)) + " files changed")

3 files changed


In [119]:
all_directories = []
for file in files:
    file_split = file.split()
    full_dir = file_split[5]
    directory = os.path.dirname(full_dir)
    all_directories.append(directory)
all_directories = removeDuplicates(all_directories)
print(str(len(all_directories)) + " directories changed")

3 directories changed


In [120]:
lines_changed = repo.git.show("--pretty=tformat:", "--numstat", vcc_commit).splitlines()
total_lines_added = 0
total_lines_deleted = 0
for line in lines_changed:
    line_split = line.split()
    total_lines_added += int(line_split[0].strip())
    total_lines_deleted += int(line_split[1].strip())
print("Total number of deletions: " + str(total_lines_deleted))
print("Total number of additions: " + str(total_lines_added))

Total number of deletions: 0
Total number of additions: 830


In [121]:
def countNumberBlankComments(file, langauge):
    total_blank_additions = 0
    total_blank_deletions = 0
    if langauge == "Java":
        for line in file:
            if line[0] == "+":
                line = line[1:]
                line = line.strip()
                if line == "":
                    total_blank_additions += 1
                    continue
                if line[0] == "/" or line[0] == "*":
                    total_blank_additions += 1
            if line[0] == "-":
                line = line[1:]
                line = line.strip()
                if line == "":
                    total_blank_deletions += 1
                    continue
                if line[0] == "/" or line[0] == "*":
                    total_blank_deletions += 1
    return total_blank_additions, total_blank_deletions

In [122]:
total_blank = [0, 0]
for file in files:
    file_split = file.split()
    full_dir = file_split[5]
    affected_file_diff = repo.git.diff(vcc_commit + "^" + ".." + vcc_commit, "--", full_dir).splitlines()
    this_total_blank = countNumberBlankComments(affected_file_diff, "Java")  
    total_blank = np.add(total_blank, this_total_blank)
total_blank_additions = total_lines_added - total_blank[0]
total_blank_deletions = total_lines_deleted - total_blank[1]
print("Total number of deletions (without comments/blanklines): " + str(total_blank_deletions))
print("Total number of additions (without comments/blanklines): " + str(total_blank_additions))

Total number of deletions (without comments/blanklines): 0
Total number of additions (without comments/blanklines): 603


In [123]:
def convertToDate(input_date):
    split = input_date.split()
    year = split[4]
    month = strptime(split[1],'%b').tm_mon
    day = split[2]
    return date(year = int(year), month = month, day = int(day))

In [124]:
for affected_file in affected_files:
    dates = repo.git.log("--format=%ad", vcc_commit, "--", affected_file).splitlines()
    if len(dates) <= 0:
        print("File not in this commit: ", affected_file)
        continue
    date1_object = convertToDate(dates[0])
    if len(dates) < 2:
        print("No changes made")
    else:    
        date2_object = convertToDate(dates[1])
        total_days = abs((date1_object - date2_object).days)
        print("Total days since file", os.path.basename(affected_file), "changed:", total_days)

No changes made


In [125]:
for affected_file in affected_files:
    times = repo.git.log("--format=%ad", vcc_commit, "--", affected_file).splitlines()
    print("This file", os.path.basename(affected_file), "has been changed:", len(times), "times")

This file IptcAnpaParser.java has been changed: 1 times


In [126]:
how_many = repo.git.shortlog("-ns","--all","--no-merges",vcc_commit,"--",affected_files).splitlines()
print("For file")
for line in how_many:
    print(line)

For file
    10	Nick Burch
     3	Tyler Palsulich
     3	tballison
     1	Bob Paulin
     1	Ray Gauss II
     1	Tim Allison
